In [3]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import date, timedelta
from config import password

In [6]:
#importing all csv datasets
ny_une = "csv/original/unemploymentnycounty.csv"
cal_une = "csv/original/unemploymentcalcounty.csv"
fl_une = "csv/original/FLunemploymentcountyjune.csv"

In [7]:
#changing csvs to panda dataframes
ny_df = pd.read_csv(ny_une)
fl_df = pd.read_csv(fl_une)
cal_df = pd.read_csv(cal_une)

In [13]:
#create NY df and clean it
ny_df2 = ny_df.rename(columns = {"RANK" : 'Unemployment Rank', "COUNTY":  'County', "RATE": 'Unemployment_Rate'})
ny_df2 = ny_df2.drop(columns = ['Unemployment Rank'])
ny_df2['Year'] = "2020"
ny_df2['Month'] = "June"
ny_df2["State"] = "NY"
ny_df2.reset_index()
ny_final = ny_df2.set_index('Month')
ny_final.dropna()
#checking to make sure all county values are unique and not duplitcates
ny_final.County.nunique()
ny_final.County.count()
# ny_final.to_csv('ny_final.csv')

62

In [18]:
fl_df.head()
#clean up columns
fl_df2 = fl_df.drop(['LABOR', 'EMPLOY-','UNEMPLOYMENT'], axis=1)
fl_df3 = fl_df2.rename(columns = {"Unnamed: 0": "County", "Unnamed: 4": "Unemployment_Rate"})
fl_df_final = fl_df3.drop([0, 0])
fl_df_final.reset_index()
#set columns to clarify month and year
fl_df_final["Year"] = "2020"
fl_df_final["Month"] = "June"
fl_df_final["State"] = "FL"
fl_final = fl_df_final.set_index('Month')
fl_final.dropna()


fl_final['Unemployment_Rate'] = fl_final['Unemployment_Rate'].str.rstrip('%').astype('float') 
#checking to make sure all county values are unique and not duplitcates
fl_final.County.nunique()
fl_final.County.count()
# fl_final.to_csv('fl_final.csv')

73

In [30]:
cal_df.head()
cal_df2 = cal_df.drop(columns = ['Area Type ','Seasonally Adjusted (Y/N) ','Status (Preliminary / Final) ','Unemployment ','Employment ','Labor Force'])
cal_df2['Date'] = pd.to_datetime(cal_df2['Date'])

#get only month of june
res = cal_df2[~(cal_df2['Date'] <= '2020-05-31')]
#continue cleaning: drop only 'california' from country and renaming columns.
cal_df_final = res
cal_df_final
cal_tidy = cal_df_final.rename(columns = {'Area Name ': 'County_CA', 'Unemployment Rate': 'Unemployment_Rate'}, inplace = False)
cal_final= cal_tidy[cal_tidy.County_CA != 'California']
cal_final = cal_final.rename(columns = {'County_CA': 'County, CA'}, inplace = False)
#fix County column so only county name appears
cal_final = cal_final.drop(columns = ['Date'])
cal_final["County, CA"]= cal_final["County, CA"].str.split(",", n = 1, expand = True)
cal_final["State"] = "CA"
cal_final = cal_final.rename(columns = {'County, CA':  'County'}, inplace= False)
cal_final = cal_final.set_index('Month')
cal_final = cal_final.rename(columns = {'Unemployment Rate ': 'Unemployment_Rate'})

cal_final.dropna()
# cal_final.to_csv('cal_final.csv')
list_of_counties = cal_final['County'].to_list()
#checking to make sure all county values are unique and not duplitcates (California DID have duplicate values due to a CDP designation, upon further research this is
#due to the Census program in California)
cal_final.County.nunique()
cal_final.County.count()

2072

In [7]:

#create table that contains all information on NY, FL, and CA
final = pd.merge(cal_final,fl_final, on ='Month', how='outer')

concatenated = pd.concat([cal_final, fl_df_final, ny_df2])


concatenated["Month"] = "June"
concatenated["Year"] = "2020"
concatenated.set_index('Month')
df = concatenated.copy()
# df.to_csv('fulllist.csv')
df


,County,Year,Unemployment_Rate,State,Year,Month
June,Alameda County,2020.0,0.135,CA,2020,June
June,Alpine County,2020.0,0.189,CA,2020,June
June,Amador County,2020.0,0.127,CA,2020,June
June,Butte County,2020.0,0.122,CA,2020,June
June,Calaveras County,2020.0,0.11,CA,2020,June
...,...,...,...,...,...,...
67,New York County,NaN,16,NY,2020,June
68,Richmond County,NaN,18.1,NY,2020,June
69,Kings County,NaN,20.5,NY,2020,June
70,Queens County,NaN,21.8,NY,2020,June


In [13]:
#connect to postgres
engine = create_engine(password)
connection = engine.connect()

In [14]:
engine.table_names()

['ca', 'base_table', 'fl', 'ny']

In [67]:
fl = pd.read_sql_query('select * from fl', con=engine).head()
fl.loc[fl['unemployment_rate'] > 7.0]

,month,county,unemployment_rate,year,state,id_fl
4,June,Brevard County,8.2,2020,FL,5


In [71]:
ny = pd.read_sql_query('select * from ny', con=engine).head()
ny.loc[fl['unemployment_rate'] >= 7.0]

,month,county,unemployment_rate,year,state,id_ny
3,June,Columbia County,8.8,2020,NY,4
4,June,Tompkins County,8.9,2020,NY,5


In [60]:
ca = pd.read_sql_query('select * from ca', con=engine).head()
ca
#example query
ca.loc[ca['unemployment_rate'] >= 10.0]


,month,county,year,unemployment_rate,state,id_cali
0,June,Alameda County,2020.0,13.5,CA,1
1,June,Alpine County,2020.0,18.9,CA,2
2,June,Amador County,2020.0,12.7,CA,3
3,June,Butte County,2020.0,12.2,CA,4
4,June,Calaveras County,2020.0,11.0,CA,5


In [61]:
base = pd.read_sql_query('select * from base_table', con=engine).head()
base.loc[ca['unemployment_rate'] >= 16.0]

,year,month,state,county,unemployment_rate,id_unemployment
1,2020,June,CA,Alpine County,18.9,2
